In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torchvision import models

# Define the transforms
trans = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
])

# Load the dataset
cifar_train = datasets.CIFAR10('cifar', train=True, transform=trans,download=True)
cifar_test = datasets.CIFAR10('cifar', train=False, transform=trans,download=True)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:

# Define the data loaders
trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=64, shuffle=False)

# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = models.resnet18(pretrained=False)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))